# 第 11 章：时间序列预测

> 用 RNN/LSTM 预测股票、天气、销量等时序数据

---

## 本章目标

- [ ] 理解时间序列预测的特点
- [ ] 掌握数据预处理（滑动窗口、标准化）
- [ ] 实现单步和多步预测
- [ ] 评估预测性能（MAE, RMSE, MAPE）

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
np.random.seed(42)

---

## 第一部分：数据准备

### 1.1 生成合成时间序列

In [ ]:
def generate_time_series(n_points=1000):
    """
    生成包含趋势、季节性和噪声的时间序列
    """
    t = np.arange(n_points)
    
    # 趋势
    trend = 0.02 * t
    
    # 季节性（周期 = 50）
    seasonal = 10 * np.sin(2 * np.pi * t / 50)
    
    # 噪声
    noise = np.random.randn(n_points) * 2
    
    series = trend + seasonal + noise + 50
    return series


# 生成数据
data = generate_time_series(1000)

# 可视化
plt.figure(figsize=(14, 4))
plt.plot(data)
plt.title('合成时间序列')
plt.xlabel('时间')
plt.ylabel('值')
plt.grid(True, alpha=0.3)
plt.show()

print(f"数据形状: {data.shape}")
print(f"范围: [{data.min():.2f}, {data.max():.2f}]")

### 1.2 滑动窗口数据集

In [ ]:
def create_sequences(data, seq_len, pred_len=1):
    """
    创建滑动窗口序列
    
    参数:
        data: 原始序列
        seq_len: 输入序列长度
        pred_len: 预测长度
    """
    X, y = [], []
    for i in range(len(data) - seq_len - pred_len + 1):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len:i+seq_len+pred_len])
    return np.array(X), np.array(y)


# 标准化
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.reshape(-1, 1)).flatten()

# 创建序列
SEQ_LEN = 30
X, y = create_sequences(data_scaled, SEQ_LEN)

# 划分训练/测试集
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# 转为 tensor
X_train = torch.FloatTensor(X_train).unsqueeze(-1).to(device)
y_train = torch.FloatTensor(y_train).to(device)
X_test = torch.FloatTensor(X_test).unsqueeze(-1).to(device)
y_test = torch.FloatTensor(y_test).to(device)

print(f"训练集: X={X_train.shape}, y={y_train.shape}")
print(f"测试集: X={X_test.shape}, y={y_test.shape}")

---

## 第二部分：模型实现

In [ ]:
class LSTMForecaster(nn.Module):
    """
    LSTM 时间序列预测模型
    """
    
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                           batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 只取最后一个时间步
        return out


# 创建模型
model = LSTMForecaster(hidden_size=64, num_layers=2).to(device)
print(model)

### 2.1 训练

In [ ]:
def train_model(model, X_train, y_train, n_epochs=100, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    losses = []
    
    for epoch in range(n_epochs):
        model.train()
        optimizer.zero_grad()
        
        pred = model(X_train)
        loss = criterion(pred, y_train)
        
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        
        if (epoch + 1) % 20 == 0:
            print(f"Epoch {epoch+1}: Loss = {loss.item():.6f}")
    
    return losses


losses = train_model(model, X_train, y_train, n_epochs=100)

### 2.2 评估

In [ ]:
def evaluate_model(model, X_test, y_test, scaler):
    model.eval()
    with torch.no_grad():
        pred = model(X_test)
    
    # 反标准化
    pred_np = pred.cpu().numpy()
    y_np = y_test.cpu().numpy()
    
    pred_orig = scaler.inverse_transform(pred_np)
    y_orig = scaler.inverse_transform(y_np)
    
    # 计算指标
    mae = np.mean(np.abs(pred_orig - y_orig))
    rmse = np.sqrt(np.mean((pred_orig - y_orig)**2))
    
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    
    # 可视化
    plt.figure(figsize=(14, 5))
    plt.plot(y_orig[:100], label='真实值', linewidth=2)
    plt.plot(pred_orig[:100], label='预测值', linewidth=2, linestyle='--')
    plt.legend()
    plt.title('时间序列预测结果')
    plt.grid(True, alpha=0.3)
    plt.show()


evaluate_model(model, X_test, y_test, scaler)

---

## 本章总结

1. **数据预处理**：标准化 + 滑动窗口
2. **模型**：LSTM/GRU 适合捕获时序依赖
3. **评估指标**：MAE, RMSE, MAPE
4. **注意事项**：避免数据泄露，正确划分训练/测试集

---

## 下一章预告

**第 12 章：综合实战项目**

👉 [12_rnn_projects.ipynb](./12_rnn_projects.ipynb)